In [1]:
import pyspark
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession

# from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ML_Models").getOrCreate()
# spark.conf.set("spark.executor.memory", "8g")
# spark.conf.set("spark.executor.cores", "4")
# spark.conf.set("spark.driver.cores", "4")
# conf = SparkConf().set("spark.jars", "/Users/sz904/Downloads/postgresql-42.2.18.jar")
# sc = SparkContext( conf=conf)



22/11/28 19:51:22 WARN Utils: Your hostname, STEVENs-MacBook-Pro-4.local resolves to a loopback address: 127.0.0.1; using 172.25.2.158 instead (on interface en0)
22/11/28 19:51:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/28 19:51:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/28 19:51:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/11/28 19:51:23 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/11/28 19:51:23 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
col_names = [
    'tcp_flags',
    'tcp_time_delta',
    'tcp_len',
    'mqtt_conack_flags',
    'mqtt_conack_flags_reserved',
    'mqtt_conack_flags_sp',
    'mqtt_conack_val',
    'mqtt_conflag_cleansess',
    'mqtt_conflag_passwd',
    'mqtt_conflag_qos',
    'mqtt_conflag_reserved',
    'mqtt_conflag_retain',
    'mqtt_conflag_uname',
    'mqtt_conflag_willflag',
    'mqtt_conflags',
    'mqtt_dupflag',
    'mqtt_hdrflags',
    'mqtt_kalive',
    'mqtt_len',
    'mqtt_msg',
    'mqtt_msgid',
    'mqtt_msgtype',
    'mqtt_proto_len',
    'mqtt_protoname',
    'mqtt_qos',
    'mqtt_retain',
    'mqtt_sub_qos',
    'mqtt_suback_qos',
    'mqtt_ver',
    'mqtt_willmsg',
    'mqtt_willmsg_len',
    'mqtt_willtopic',
    'mqtt_willtopic_len',
    'target',
    'dataset',
    'tcp_flags_decimal',
    'mqtt_conflags_decimal',
    'mqtt_hdrflags_decimal'
]

nominal_cols = ["mqtt_msg"]

corelated_cols_to_remove = ["mqtt_conack_flags",
                     "mqtt_conack_flags_reserved",
                     "mqtt_conack_flags_sp", 
                     "mqtt_conflag_qos", 
                     "mqtt_conflag_reserved", 
                     "mqtt_conflag_retain", 
                     "mqtt_conflag_willflag", 
                     "mqtt_willtopic", 
                     "mqtt_willtopic_len", 
                     "mqtt_sub_qos", 
                     "mqtt_suback_qos", 
                     "mqtt_willmsg",
                     "mqtt_willmsg_len"]



binary_cols = ["mqtt_conack_val", 
                  "mqtt_conflag_cleansess", 
                  "mqtt_conflag_passwd", 
                  "mqtt_conflag_uname", 
                  "mqtt_dupflag", 
                  "mqtt_proto_len", 
                  "mqtt_protoname", 
                  "mqtt_qos", 
                  "mqtt_retain", 
                  "mqtt_ver"]


continuous_cols = ["tcp_flags_decimal",
                     'tcp_time_delta',
                     'tcp_len',
                     'mqtt_kalive',
                     'mqtt_len',
                     'mqtt_msgid',
                     'mqtt_msgtype',
                     'mqtt_conflags_decimal', 
                     "mqtt_hdrflags_decimal", ]

In [3]:
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.ml import Pipeline,Transformer
from pyspark.ml.feature import Imputer,StandardScaler,StringIndexer,OneHotEncoder, VectorAssembler

from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

import numpy as np
from matplotlib import pyplot as plt


class OutcomeCreater(Transformer): # this defines a transformer that creates the outcome column
    
    def __init__(self):
        super().__init__()

    def _transform(self, dataset):
        # label_to_binary = udf(lambda name: 0.0 if name == 'normal' else 1.0)
        # output_df = dataset.withColumn('outcome', label_to_binary(col('class'))).drop("class")  
        # output_df = output_df.withColumn('outcome', col('outcome').cast(DoubleType()))
        # output_df = output_df.drop('difficulty')
        # return output_df

        def converter(name):
            if name == 'legitimate':
                return 0
            elif name == 'dos':
                return 1      
            elif name == 'malformed':
                return 2     
            elif name == 'flood':
                return 3
            elif name == 'bruteforce':
                return 4   
            elif name == 'slowite':
                return 5  
            else:
                print("out of range")
                return None
            
        label_to_binary = udf(lambda name: converter(name))
        output_df = dataset.withColumn('label', label_to_binary(col('target')))
        output_df = output_df.drop("target")  
        output_df = output_df.withColumn('label', col('label').cast(DoubleType()))
        return output_df

class FeatureTypeCaster(Transformer): # this transformer will cast the columns as appropriate types  
    def __init__(self):
        super().__init__()

    def _transform(self, dataset):
        output_df = dataset
        for col_name in binary_cols + continuous_cols:
            output_df = output_df.withColumn(col_name,col(col_name).cast(DoubleType()))

        return output_df
class ColumnDropper(Transformer): # this transformer drops unnecessary columns
    def __init__(self, columns_to_drop = None):
        super().__init__()
        self.columns_to_drop=columns_to_drop
    def _transform(self, dataset):
        output_df = dataset
        for col_name in self.columns_to_drop:
            output_df = output_df.drop(col_name)
        return output_df



In [4]:
train_path = "df_train_pandas.csv"
test_path = "df_test_pandas.csv"
df_train_updated = spark.read.csv(train_path, header=True, inferSchema= True).toDF(*col_names)
df_test_updated = spark.read.csv(test_path, header=True, inferSchema= True).toDF(*col_names)
print(df_train_updated.count())
print(df_test_updated.count())


231646
99290


In [5]:
df_train_updated = df_train_updated.drop("tcp_flags")
df_train_updated = df_train_updated.drop("mqtt_conflags")
df_train_updated = df_train_updated.drop("mqtt_hdrflags")
df_train_updated = df_train_updated.drop("dataset")

df_test_updated = df_test_updated.drop("tcp_flags")
df_test_updated = df_test_updated.drop("mqtt_conflags")
df_test_updated = df_test_updated.drop("mqtt_hdrflags")
df_test_updated = df_test_updated.drop("dataset")


df_train_updated = df_train_updated.drop("mqtt_msg")
df_test_updated = df_test_updated.drop("mqtt_msg")

df_train_updated = df_train_updated.drop("mqtt_conack_flags")
df_test_updated = df_test_updated.drop("mqtt_conack_flags")

df_train_updated = df_train_updated.drop("mqtt_protoname")
df_test_updated = df_test_updated.drop("mqtt_protoname")

In [6]:
outcomeCreate = OutcomeCreater()
stage_vector_assembler = VectorAssembler(inputCols=[column for column in df_train_updated.columns if column != 'target'], outputCol="vectorized_features")
stage_scaler = StandardScaler(inputCol= 'vectorized_features', outputCol= 'features')
len(stage_vector_assembler.getInputCols())


30

In [7]:
df_train = outcomeCreate.transform(df_train_updated)
df_train = stage_vector_assembler.transform(df_train)
scaler_model = stage_scaler.fit(df_train)
df_train = scaler_model.transform(df_train)
# df_train.printSchema()

22/11/28 19:52:04 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [8]:
df_test = outcomeCreate.transform(df_test_updated)
df_test = stage_vector_assembler.transform(df_test)
df_test = scaler_model.transform(df_test)
# df_test.printSchema()

In [9]:
dropper = ColumnDropper(
    [column for column in df_train.columns if column != 'label' and column != 'features']
)

df_train = dropper.transform(df_train)
df_test = dropper.transform(df_test)
df_train.printSchema()


root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



------------------Pyspark ML- Logistic Regression------------------------

In [10]:
# logistic model
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label')
#### default regparam 0, default maxIter = 100. 

In [11]:
lr_model = lr.fit(df_train)
evaluator = MulticlassClassificationEvaluator(
    predictionCol='prediction', 
    labelCol='label', 
    metricName='accuracy'
)
lr_prediction_train = lr_model.transform(df_train)
lr_prediction_test = lr_model.transform(df_test)
lr_accuracy_train = evaluator.evaluate(lr_prediction_train)
lr_accuracy_test = evaluator.evaluate(lr_prediction_test)
print(f"Logistic Regression Train accuracy = {np.round(lr_accuracy_train*100,2)}%, test accuracy = {np.round(lr_accuracy_test*100,2)}%")


22/11/28 19:52:11 WARN InstanceBuilder$JavaBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
22/11/28 19:52:11 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/11/28 19:52:11 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


Logistic Regression Train accuracy = 80.45%, test accuracy = 80.48%


In [12]:
lr_model.getMaxIter()

100

In [13]:


# cross validation
lr_paramGrid = (ParamGridBuilder()
            .addGrid(lr.regParam, [0, 0.0001, 0.001])
            .addGrid(lr.maxIter, [ 100, 120, 150, 200])
            .build())

evaluator = MulticlassClassificationEvaluator(
    predictionCol='prediction', 
    labelCol='label', 
    metricName='accuracy'
)

lr_cv = CrossValidator(estimator=lr, estimatorParamMaps=lr_paramGrid, 
                    evaluator=evaluator, numFolds=5)


In [14]:
lr_cv_model = lr_cv.fit(df_train)
res_train = lr_cv_model.transform(df_train)
res_test = lr_cv_model.transform(df_test)


In [15]:
print(lr_cv_model.getEstimatorParamMaps()[np.argmax(lr_cv_model.avgMetrics)])


{Param(parent='LogisticRegression_4569534dad0d', name='regParam', doc='regularization parameter (>= 0).'): 0.0, Param(parent='LogisticRegression_4569534dad0d', name='maxIter', doc='max number of iterations (>= 0).'): 200}


In [16]:
accuracy_test = (res_test.filter(res_test.label == res_test.prediction)
    .count() / float(res_test.count()))
print(f"Test Accuracy : {np.round(accuracy_test*100,2)}%")

Test Accuracy : 80.48%


--------------PySpark ML Multi layer Perceptron----------------------

In [26]:
from pyspark.ml.classification import MultilayerPerceptronClassifier


In [27]:
layers = [30, 10, 20,  6]
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)

nn_model = trainer.fit(df_train)

nn_prediction_train = nn_model.transform(df_train)
nn_prediction_test = nn_model.transform(df_test)

predictionAndLabels = nn_prediction_test.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))





Test set accuracy = 0.8052573270218552


In [28]:
layers1 = [30, 10, 20,  20, 6]
trainer1 = MultilayerPerceptronClassifier(maxIter=100, layers=layers1, blockSize=128, seed=1234)

nn_model1 = trainer1.fit(df_train)

nn_prediction_train1 = nn_model1.transform(df_train)
nn_prediction_test1 = nn_model1.transform(df_test)

predictionAndLabels1 = nn_prediction_test1.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels1)))



Test set accuracy = 0.7932118038070299


In [ ]:
nn_param_grid = ParamGridBuilder().addGrid(trainer.layers,
                                           [[30, 10, 20,  6],
                                            [30, 10, 20, 20,  6],
                                            [30, 10, 20, 40, 6],
                                            [30, 10, 20, 40, 20, 6]]).build()
#  .addGrid(trainer.maxIter,[100])
nn_cv = CrossValidator(estimator=trainer,estimatorParamMaps=nn_param_grid,evaluator=evaluator,numFolds=5)


In [ ]:
nn_cv_model = nn_cv.fit(df_train)

nn_cv_prediction_train = nn_cv_model.transform(df_train)
nn_cv_prediction_test = nn_cv_model.transform(df_test)

predictionAndLabels = nn_prediction_test.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))



---------------TF ---------------------------------------------------

In [13]:
from tensorflow import keras
import tensorflow as tf  # now import the tensorflow module
print(tf.__version__)  # make sure the version is 2.x
# from pyspark.sql.functions import udf
import pandas as pd
import numpy as np

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: cannot open shared object file: No such file or directory']
  warnings.warn(f"file s

2.11.0


In [14]:
df_validate, df_test = df_test.randomSplit([0.5,0.5])
to_array = udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))


df_train_pandas = df_train.withColumn('features', to_array('features')).toPandas()
df_validate_pandas = df_validate.withColumn('features', to_array('features')).toPandas()
df_test_pandas = df_test.withColumn('features', to_array('features')).toPandas()

x_train = tf.constant(np.array(df_train_pandas['features'].values.tolist()))
y_train = tf.constant(np.array(df_train_pandas['label'].values.tolist()))

x_validate = tf.constant(np.array(df_validate_pandas['features'].values.tolist()))
y_validate = tf.constant(np.array(df_validate_pandas['label'].values.tolist()))


x_test = tf.constant(np.array(df_test_pandas['features'].values.tolist()))
y_test = tf.constant(np.array(df_test_pandas['label'].values.tolist()))

-----------------------TF shallow NN CV------------------------------

In [15]:
from sklearn.model_selection import KFold

num_folds = 5
HP_DEPTH_shallowNN_CV = 5
kfold = KFold(n_splits=num_folds, shuffle=True)
log_dir_tfShallowNN_CV =  "TFshallow_NN_CV/"
Acc_shallow = []
Loss_shallow = []

In [16]:
from tensorboard.plugins.hparams import api as hp

HP_WIDTH_shallowNN = hp.HParam('NN_width', hp.Discrete([15, 25, 35, 50]))
HP_ActivationFunc_shallowNN = hp.HParam('activation_func', hp.Discrete(['tanh', 'gelu']))

with tf.summary.create_file_writer(log_dir_tfShallowNN_CV).as_default():
    hp.hparams_config(
        hparams=[HP_WIDTH_shallowNN, HP_ActivationFunc_shallowNN],
        metrics=[hp.Metric('Accuracy')],
    )

In [25]:
def train_test_model(hparams,logdir):
    model = keras.Sequential()
    
    for _ in range(HP_DEPTH_shallowNN_CV):
        model.add(keras.layers.Dense(hparams[HP_WIDTH_shallowNN] ,activation=hparams[HP_ActivationFunc_shallowNN]))
        
    model.add(keras.layers.Dense(6))
    
    model.compile(optimizer = 'adam',
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics = [keras.metrics.SparseCategoricalAccuracy(name = 'Accuracy_epochs')])

#     callback = [tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)]
    callback = tf.keras.callbacks.EarlyStopping(monitor="val_Accuracy_epochs", patience = 10, min_delta=0.0003, restore_best_weights=True)
#     By setting restore_best_weights=True in callback, the best model is used for prediction
    history_shallowNN_CV = model.fit(x_train, y_train, epochs=30, verbose = 2,
    callbacks=callback,
    validation_data = (x_validate, y_validate))
    
    results = model.evaluate(x_test, y_test, batch_size=128)
    
    accuracy = np.max(history_shallowNN_CV.history["val_Accuracy_epochs"])
    return accuracy, results

In [26]:

for hp_activationfunc in HP_ActivationFunc_shallowNN.domain.values:
    for width in (HP_WIDTH_shallowNN.domain.values):
        hparams = {
            HP_WIDTH_shallowNN: width,
            HP_ActivationFunc_shallowNN: hp_activationfunc,
        }
        run_name = f"run-WIDTH{int(hparams[HP_WIDTH_shallowNN])}-ActivationFunction{hparams[HP_ActivationFunc_shallowNN]}"
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})

        run_dir = log_dir_tfShallowNN_CV + run_name
        accuracy, results = train_test_model(hparams,run_dir)
        print("***************************test loss, test acc:", results)

        with tf.summary.create_file_writer(run_dir).as_default():
            hp.hparams(hparams)  # record the values used in this trial
            tf.summary.scalar("Accuracy", accuracy, step=1)

--- Starting trial: run-WIDTH15-ActivationFunctiongelu
{'NN_width': 15, 'activation_func': 'gelu'}
Epoch 1/30
7239/7239 - 9s - loss: 0.4786 - Accuracy_epochs: 0.8111 - val_loss: 0.4328 - val_Accuracy_epochs: 0.8269 - 9s/epoch - 1ms/step
Epoch 2/30
7239/7239 - 8s - loss: 0.4217 - Accuracy_epochs: 0.8280 - val_loss: 0.4314 - val_Accuracy_epochs: 0.8276 - 8s/epoch - 1ms/step
Epoch 3/30
7239/7239 - 7s - loss: 0.4163 - Accuracy_epochs: 0.8283 - val_loss: 0.4292 - val_Accuracy_epochs: 0.8274 - 7s/epoch - 1ms/step
Epoch 4/30
7239/7239 - 8s - loss: 0.4143 - Accuracy_epochs: 0.8289 - val_loss: 0.4268 - val_Accuracy_epochs: 0.8294 - 8s/epoch - 1ms/step
Epoch 5/30
7239/7239 - 7s - loss: 0.4128 - Accuracy_epochs: 0.8288 - val_loss: 0.4315 - val_Accuracy_epochs: 0.8293 - 7s/epoch - 1ms/step
Epoch 6/30
7239/7239 - 7s - loss: 0.4117 - Accuracy_epochs: 0.8287 - val_loss: 0.4269 - val_Accuracy_epochs: 0.8264 - 7s/epoch - 1ms/step
Epoch 7/30
7239/7239 - 7s - loss: 0.4111 - Accuracy_epochs: 0.8289 - val_

--------------------TF Deep NN CV------------------------------------

In [17]:
from sklearn.model_selection import KFold

num_folds = 5
HP_DEPTH_DeepNN_CV = 25
kfold = KFold(n_splits=num_folds, shuffle=True)
log_dir_tfDeepNN_CV =  "TFDeep_NN_CV/"


In [ ]:
from tensorboard.plugins.hparams import api as hp

HP_WIDTH_DeepNN = hp.HParam('NN_width', hp.Discrete([10, 15, 20]))
# HP_ActivationFunc_DeepNN = hp.HParam('activation_func', hp.Discrete(['sigmoid', 'tanh']))
HP_ActivationFunc_DeepNN = hp.HParam('activation_func', hp.Discrete(['tanh', 'gelu']))

with tf.summary.create_file_writer(log_dir_tfDeepNN_CV).as_default():
    hp.hparams_config(
        hparams=[HP_WIDTH_DeepNN, HP_ActivationFunc_DeepNN],
        metrics=[hp.Metric('Accuracy')],
    )
    
    

In [23]:
def train_test_model(hparams,logdir):
    model = keras.Sequential()
    
    for _ in range(HP_DEPTH_DeepNN_CV):
        model.add(keras.layers.Dense(hparams[HP_WIDTH_DeepNN] ,activation=hparams[HP_ActivationFunc_DeepNN]))
        
    model.add(keras.layers.Dense(6))
    
    model.compile(optimizer = 'adam',
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics = [keras.metrics.SparseCategoricalAccuracy(name = 'Accuracy_epochs')])

    #     callback = [tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)]
    callback = tf.keras.callbacks.EarlyStopping(monitor="val_Accuracy_epochs", patience = 12, min_delta=0.0003, restore_best_weights=True)
    history_DeepNN_CV = model.fit(x_train, y_train, epochs=20, verbose = 2,
    callbacks=callback,
    validation_data = (x_validate, y_validate))
    
    results = model.evaluate(x_test, y_test, batch_size=128)


    accuracy = np.max(history_DeepNN_CV.history["val_Accuracy_epochs"])
    return accuracy, results


In [24]:

for hp_activationfunc in HP_ActivationFunc_DeepNN.domain.values:
    for width in (HP_WIDTH_DeepNN.domain.values):
        hparams = {
            HP_WIDTH_DeepNN: width,
            HP_ActivationFunc_DeepNN: hp_activationfunc,
        }
        run_name = f"run-WIDTH{int(hparams[HP_WIDTH_DeepNN])}-ActivationFunction{hparams[HP_ActivationFunc_DeepNN]}"
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})

        run_dir = log_dir_tfDeepNN_CV + run_name
        accuracy, results = train_test_model(hparams,run_dir)
        print("***************************test loss, test acc:", results)

        with tf.summary.create_file_writer(run_dir).as_default():
            hp.hparams(hparams)  # record the values used in this trial
            tf.summary.scalar("Accuracy", accuracy, step=1)

--- Starting trial: run-WIDTH10-ActivationFunctiongelu
{'NN_width': 10, 'activation_func': 'gelu'}
Epoch 1/20
7239/7239 - 15s - loss: 0.5297 - Accuracy_epochs: 0.7923 - val_loss: 0.4561 - val_Accuracy_epochs: 0.8114 - 15s/epoch - 2ms/step
Epoch 2/20
7239/7239 - 12s - loss: 0.4525 - Accuracy_epochs: 0.8181 - val_loss: 0.4798 - val_Accuracy_epochs: 0.8123 - 12s/epoch - 2ms/step
Epoch 3/20
7239/7239 - 12s - loss: 0.4426 - Accuracy_epochs: 0.8211 - val_loss: 0.4669 - val_Accuracy_epochs: 0.8129 - 12s/epoch - 2ms/step
Epoch 4/20
7239/7239 - 12s - loss: 0.4533 - Accuracy_epochs: 0.8165 - val_loss: 0.4569 - val_Accuracy_epochs: 0.8183 - 12s/epoch - 2ms/step
Epoch 5/20
7239/7239 - 12s - loss: 0.4458 - Accuracy_epochs: 0.8208 - val_loss: 0.4375 - val_Accuracy_epochs: 0.8245 - 12s/epoch - 2ms/step
Epoch 6/20
7239/7239 - 12s - loss: 0.4392 - Accuracy_epochs: 0.8212 - val_loss: 0.4429 - val_Accuracy_epochs: 0.8198 - 12s/epoch - 2ms/step
Epoch 7/20
7239/7239 - 12s - loss: 0.4406 - Accuracy_epochs: 